<a href="https://colab.research.google.com/github/zzhu104/MAT494_HW/blob/main/HW3.7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3.7 Neural Networks TensorFlow 2

Here we provide a simple example of Neural Network using TensorFlow 2 with MNIST dataset.

Source:

https://www.tensorflow.org/tutorials/quickstart/beginner

In [2]:
import tensorflow as tf

1. Load and prepare the MNIST dataset

Reference:

https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data



This is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. It is considered another classic dataset in machine learning area. The images are handwritings of English letters. Researchers are using it as a training set to build neural network models that would recognize human handwritings.

The dataset we import from tf.keras is not the original image set, but numerical matrix that transfromed from the images via certain kernels.

tf.keras.datasets.mnist.load_data() will return tuples of arrays:



*   x_train: uint8 NumPy array of grayscale image data with shapes (60000, 28, 28), containing the training data. Pixel values range from 0 to 255.

*   y_train: uint8 NumPy array of digit labels (integers in range 0-9) with shape (60000,) for the training data.

*   x_test: uint8 NumPy array of grayscale image data with shapes (10000, 28, 28), containing the test data. Pixel values range from 0 to 255.


*   y_test: uint8 NumPy array of digit labels (integers in range 0-9) with shape (10000,) for the test data.



In [3]:
# load the dataset
mnist = tf.keras.datasets.mnist

# convert the samples from integers to floating-point numbers
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


2. Build the tf.keras.Sequential model by stacking layers.

Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.33594853,  0.5768234 ,  0.2460109 , -0.28828022,  0.16308455,
        -0.3064571 , -0.15035203, -0.78881264, -0.6313912 , -0.05889017]],
      dtype=float32)

The tf.nn.softmax function converts these logits to "probabilities" for each class:

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.07746037, 0.19297071, 0.13861856, 0.0812422 , 0.12758714,
        0.07977881, 0.09325735, 0.04924965, 0.05764617, 0.10218908]],
      dtype=float32)

Note: It is possible to bake this tf.nn.softmax in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

The losses.SparseCategoricalCrossentropy loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.5284972

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The Model.fit method adjusts the model parameters to minimize the loss:

In [10]:

model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2978 - accuracy: 0.9125
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1455 - accuracy: 0.9559
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1087 - accuracy: 0.9671
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0900 - accuracy: 0.9723
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0764 - accuracy: 0.9765


3. Evaluate the Model

The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0785 - accuracy: 0.9747


[0.07847832888364792, 0.9746999740600586]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.53373109e-08, 2.15711651e-07, 2.50056837e-06, 7.84898439e-05,
        1.69179795e-10, 1.01681552e-08, 6.17237208e-12, 9.99914169e-01,
        1.61352922e-08, 4.63028027e-06],
       [4.54686528e-07, 3.65603482e-04, 9.99399543e-01, 2.21455339e-04,
        4.56924842e-14, 4.89922968e-06, 1.56267004e-06, 2.91791482e-12,
        6.48195009e-06, 3.38669820e-13],
       [7.57932185e-07, 9.99348342e-01, 9.88376414e-05, 4.27479108e-06,
        3.82372637e-05, 1.08306367e-05, 2.20761522e-05, 3.61802784e-04,
        1.14082250e-04, 7.44979900e-07],
       [9.99577701e-01, 2.57085162e-07, 1.09671724e-04, 4.77692254e-07,
        2.03219383e-06, 4.87721409e-05, 1.48994019e-04, 2.74450963e-06,
        2.30292102e-07, 1.09072906e-04],
       [4.80449808e-06, 8.78516389e-08, 1.00737107e-05, 4.64002845e-08,
        9.99362528e-01, 8.82010681e-07, 1.32244659e-05, 1.32551231e-05,
        2.10620237e-06, 5.92996075e-04]], dtype=float32)>

Copyright 2019 The TensorFlow Authors.

In [14]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.